In [62]:
import numpy as np
import pandas as pd
import string
import re
from fuzzywuzzy import fuzz

<h2>Reading in the datasets</h2>

In [2]:
dna = pd.read_csv("../data/working/validcompaniesdictionary.csv", index_col = [0])

In [3]:
fda = pd.read_excel("../data/original/fda_companies.xlsx")

In [316]:
ndc = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

<h2>Neil's code for cleaning</h2>

In [5]:
removeset=string.punctuation
removeset=removeset.replace("-","") #Don't remove dashes
removeset=removeset.replace("&","") #Don't remove ampersand
removeset=removeset.replace("_","") #Don't remove underscore
removeset=removeset.replace("%","") #Don't remove percent
removeset=removeset.replace("$","") #Don't remove dollar
   
print(removeset)

!"#'()*+,./:;<=>?@[\]^`{|}~


In [183]:
# remove all single characters (This step is done first, because later there are single chars we want to retain.)
#document = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(X[sen]))
string = "hello i world"
string = re.sub(r'\s+[a-zA-Z]\s+', ' ', string)
print(string)

# remove all numbers
#document = re.sub(r'[0-9]','', document)
string2 = "h3ll0"
string2 = re.sub(r'[0-9]','', string)
print(string2)

# Substituting multiple spaces with single space
#document = re.sub(r'\s+', ' ', document, flags=re.I)
string3 = "hello      world"
string3 = re.sub(r'\s+', ' ', string3, flags=re.I)
print(string3)

#Converting to lowercase
string4 = "HEllo WORLD"
string4 = string4.lower()
print(string4)

#Removing prefixed 'b'
#document = re.sub(r'^b\s+', '', document)
string5 = "b hello world"
string5 = re.sub(r'^b\s+', '', string5)
print(string5)

#Make dashes into combined words
#document = re.sub(r'\s-\s+', '-', document)
string6 = "hello - world"
string6 = re.sub(r'\s-\s+', '-', string6)
print(string6)

#Make ampersand into combined words
#document = re.sub(r'\s&\s+', '&', document)
string7 = "hel & lo & world"
string7 = re.sub(r'\s&\s+', '&', string7)
print(string7)

#Make underscore into combined words
#document = re.sub(r'\s_\s+', '_', document)
string8 = "hel _ lo wo _ rld"
string8 = re.sub(r'\s_\s+', '_', string8)
print(string8)

hello world
hello world
hello world
hello world
hello world
hello-world
hel&lo&world
hel_lo wo_rld


In [77]:
#removes all punctuation in string that is in removeset
document = "Johnson+;Johnson!"
for i in removeset:
    document=re.sub(re.escape(i),"",document)
print(document)

JohnsonJohnson


<h2>Cleaning NDC</h2>

<h4>Removing the first 25 since they are just numbers</h4>

In [317]:
#Getting rid of the first 25 since those are just numbers
ndc = ndc.iloc[25:]
#renaming column
ndc = ndc.rename(columns = {'Row Labels':'company'})

<h4>Lowercase everything</h4>

In [318]:
#Converting to lower first
ndc.company = ndc.company.str.lower()

<h4>Remove content that are in parentheses</h4>

In [319]:
#Function for removing parentheses content
def removeParenthesesContent(string):
    return re.sub(r'\([^)]*\)', '', string)


In [320]:
ndc['companiesWithoutParensContent'] = ndc['company'].apply(removeParenthesesContent)

In [321]:
del ndc['company']

In [322]:
ndc.head()

,companiesWithoutParensContent
25,spironolactone 2%
26,-l'oreal usa products inc
27,.cardinal health
28,.church & dwight canada corp
29,{preferred pharmaeutials inc.


<h4>Remove Unwanted Punctuation</h4>

In [323]:
"""
Ignore 
#Function that uses regex to remove parentheses and square brackets
def removeParenthesis(string):
    return re.sub('[()\{}]', '', string)
"""
#function that gets rid of unwanted punctuation
#This does get rid of ' within a string (ex. l'oreal becomes l oreal) so maybe recheck?
def removeUnwantedPunc(string):
    return re.sub('[!"#\'()*+,./:;<=>?@[\]^`{|}~]', '', string)

In [324]:
ndc['companiesWithoutUnwantedPunc'] = ndc['companiesWithoutParensContent'].apply(removeUnwantedPunc)

In [325]:
del ndc['companiesWithoutParensContent']

In [326]:
ndc.rename(columns = {"companiesWithoutUnwantedPunc": "company"}, inplace = True)

In [327]:
#At this point all the companies are lowercased, don't have parenthetical content, and don't have unwanted punctuation
ndc.head()

,company
25,spironolactone 2%
26,-loreal usa products inc
27,cardinal health
28,church & dwight canada corp
29,preferred pharmaeutials inc


<h4>Removing numbers</h4>

In [328]:
#Function that will remove numbers
def removeNumbers(string):
    return re.sub(r'[0-9+]','', string)

In [329]:
ndc['companiesWithNoNumbers'] = ndc.company.apply(removeNumbers)

In [330]:
del ndc['company']

In [331]:
ndc = ndc.rename(columns = {"companiesWithNoNumbers": 'company'})

In [332]:
ndc.head()

,company
25,spironolactone %
26,-loreal usa products inc
27,cardinal health
28,church & dwight canada corp
29,preferred pharmaeutials inc


<h4>Removing Single Chars</h4>

In [333]:
def removeSingleChar(string):
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', string)

In [334]:
ndc['companiesWithNoSingleChar'] = ndc.company.apply(removeSingleChar)

In [335]:
del ndc['company']
ndc = ndc.rename(columns={"companiesWithNoSingleChar": 'company'})

<h4>Substituting Multiple Spaces with Single Space</h4>

In [336]:
def subMultipleSpacesForOne(string):
    return re.sub(r'\s+', ' ', string, flags=re.I)

In [337]:
ndc['noMultipleSpaces'] = ndc.company.apply(subMultipleSpacesForOne)

In [338]:
del ndc['company']
ndc = ndc.rename(columns = {"noMultipleSpaces": 'company'})

<h4>Removing Prefix b</h4>

In [339]:
def removePrefix(string):
    return re.sub(r'^b\s+', '', string)

In [340]:
ndc['removedPrefix'] = ndc.company.apply(removePrefix)

In [341]:
ndc.columns

Index(['company', 'removedPrefix'], dtype='object')

In [342]:
del ndc['company']
ndc = ndc.rename(columns = {"removedPrefix":"company"})

<h4>Make dashes into combined words</h4>

In [343]:
#start here
def makeDashCombined(string):
    return re.sub(r'\s-\s+', '-', string)

In [344]:
ndc['combinedDash'] = ndc.company.apply(makeDashCombined)

In [345]:
del ndc['company']
ndc = ndc.rename(columns = {"combinedDash": 'company'})

<h4>Combine Ampersand</h4>

In [346]:
def combineAmpersand(string):
    return re.sub(r'\s&\s+', '&', string)

In [347]:
ndc['combineAmp'] = ndc.company.apply(combineAmpersand)

In [348]:
ndc.columns

Index(['company', 'combineAmp'], dtype='object')

In [349]:
del ndc['company']
ndc = ndc.rename(columns = {"combineAmp": 'company'})

In [350]:
ndc.columns

Index(['company'], dtype='object')

<h4>Combine '_'</h4>

In [351]:
def combinedUnderScore(string):
    return re.sub(r'\s_\s+', '_', string)

In [352]:
ndc.columns

Index(['company'], dtype='object')

In [353]:
ndc['combinedUnder'] = ndc.company.apply(combinedUnderScore)

In [355]:
del ndc['company']
ndc = ndc.rename(columns = {"combinedUnder": 'company'})

<h2>End Initial Cleaning</h2>

<h4>Grabbing the list of legal entities from os github</h4>

In [357]:
legalEntities = pd.read_csv("https://raw.githubusercontent.com/DSPG-Young-Scholars-Program/dspg20oss/danBranch/ossPy/keyFiles/curatedLegalEntitesRaw.csv", quotechar = "'",header = None)
legalEntities.head()

,0
0,(?i) Inc\b
1,(?i) Ltd\b
2,(?i) LLC\b
3,(?i) GmbH\b
4,(?i) Corporation\b


<h4>Using Daniel's code to get remove legal entities</h4>

In [358]:
def eraseFromColumn(inputColumn, eraseList):
    "iteratively delete regex query matches from input list"
    
    """
    inputColumn -- a column from a pandas dataframe, this will be the set of
    target words/entries that deletions will be made from
    eraseList -- a column containing strings (regex expressions) which will be
    deleted from the inputColumn, in an iterative fashion
    """
    eraseList['changeNum'] = 0
    eraseList['changeIndexes'] = ''
    
    inputColumn = inputColumn.replace(regex=True, to_replace = "\\\\", value='/')
    
    for index, row in eraseList.iterrows():
        curReplaceVal = row[0]
        currentRegexExpression=re.compile(curReplaceVal)
        CurrentBoolVec=inputColumn.str.contains(currentRegexExpression, na= False)
        eraseList['changeIndexes'].iloc[index]=[i for i, x in enumerate(CurrentBoolVec) if x]
        eraseList['changeNum'].iloc[index] = len(eraseList['changeIndexes'].iloc[index])
        inputColumn.replace(regex=True, to_replace=currentRegexExpression,value='', inplace = True)
    
    return inputColumn, eraseList

In [359]:
output, eraseList = eraseFromColumn(ndc.company, legalEntities)

/home/itm3f/.local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [360]:
#This dataframe is lowercased, doesn't have any of the unwanted punctuation, and has removed all legal entity labels
output= pd.DataFrame(output)

<h4>Counting Unique String Tokens using Daniel's code</h4>

In [369]:
ndc_cleaned = output

In [371]:
longString = ndc_cleaned['company'].str.cat(sep = " ")

In [372]:
longStringSeperated = longString.split(' ')

In [373]:
uniqueSubTokenFrame = pd.DataFrame(longStringSeperated)

In [374]:
columnUniqueCounts = uniqueSubTokenFrame.iloc[:,0].value_counts()

In [375]:
tableUniqueCounts = columnUniqueCounts.reset_index()

In [376]:
tableUniqueCounts.rename(columns = {0: "count", "index":"token"}, inplace = True)

In [397]:
top20 = tableUniqueCounts.head(20).token.tolist()

In [398]:
top20

['pharmaceuticals',
 'medical',
 'products',
 'laboratories',
 'pharma',
 'anda',
 'supply',
 'health',
 'pharmaceutical',
 'usa',
 'international',
 'care',
 'and',
 'nda',
 'coltd',
 'the',
 'home',
 'healthcare',
 '',
 'of']

In [400]:
tableUniqueCounts.head(10)

,token,count
0,pharmaceuticals,549
1,medical,318
2,products,254
3,laboratories,222
4,pharma,199
5,anda,192
6,supply,192
7,health,172
8,pharmaceutical,164
9,usa,156


In [394]:
og = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

In [395]:
og = og.iloc[25:]

In [396]:
og.head(50)

,Row Labels
25,SPIRONOLACTONE 2%
26,-L'Oreal USA Products Inc
27,.Cardinal Health
28,.Church & Dwight Canada Corp
29,{Preferred Pharmaeutials INC.
30,"†Wal-Mart Stores, Inc.†"
31,1 Veterans Health
32,111 Medco
33,"1st Class Pharmaceuticals, Inc."
34,1ST MEDX LLC
